In [1]:
import os
import gc
import random
import itertools
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import tokenizers
import transformers
from transformers import AutoTokenizer, AutoConfig, AutoModel, T5EncoderModel, get_linear_schedule_with_warmup
import datasets
from datasets import load_dataset, load_metric
import sentencepiece
import argparse
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.nn as nn
from torch.optim import AdamW
import pickle
import time
import math
from sklearn.preprocessing import MinMaxScaler
from datasets.utils.logging import disable_progress_bar
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
disable_progress_bar()

def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--data_path", type=str, required=False)
#     parser.add_argument("--dataset_name", type=str, required=False)
    parser.add_argument("--pretrained_model_name_or_path", type=str, default="sagawa/ZINC-t5", required=False)
    parser.add_argument("--model_name_or_path", type=str, required=False)
    parser.add_argument("--scaler_path", type=str, default="/data2/sagawa/tcrp-regression-model-archive/10-23-1st-new-metric-reactant-product", required=False)
    parser.add_argument("--debug", action='store_true', default=False, required=False)
    parser.add_argument("--batch_size", type=int, default=5, required=False)
    parser.add_argument("--max_len", type=int, default=512, required=False)
    parser.add_argument("--num_workers", type=int, default=1, required=False)
    parser.add_argument("--fc_dropout", type=float, default=0.1, required=False)
    parser.add_argument("--output_dir", type=str, default='./', required=False)
    parser.add_argument("--seed", type=int, default=42, required=False)

    return parser.parse_args()

class CFG():
    data_path='../../all_ord_reaction_uniq_with_attr_v3.tsv'
#     pretrained_model_name_or_path = 'sagawa/ZINC-t5'
    model = 'sagawa/ZINC-t5'
    batch_size = 5 #max_lenを大きくしたらoomしたから15から5に
    seed = 42
    num_workers = 4
    output_dir = './'
    model_name_or_path = '/data2/sagawa/tcrp-regression-model-archive/10-23-1st-new-metric-reactant-product'
    scaler_path = '/data2/sagawa/tcrp-regression-model-archive/10-23-1st-new-metric-reactant-product'

In [2]:
train_ds = pd.read_csv('../../regression-input-train.csv')
train_ds

,CATALYST,REACTANT,REAGENT,SOLVENT,INTERNAL_STANDARD,NoData,PRODUCT,YIELD,TEMP,input
0,,CC(=O)Cl.CC(C)(C)OC(=O)N1CCC2(CC1)CC(=O)c1nn(C...,,CO,,,Cc1cc(Cc2cc(C)c(O)c(C=O)c2)cc(C=O)c1O,96.0,0.0,REACTANT:CC(=O)Cl.CC(C)(C)OC(=O)N1CCC2(CC1)CC(...
1,Cl[Pd](Cl)([P](c1ccccc1)(c1ccccc1)c1ccccc1)[P]...,CCCC[Sn](CCCC)(CCCC)c1cccs1.CCCCc1nc(C)c(Br)c(...,,CCOC(C)=O.CN(C)C=O,,,CCCCCCCC/C=C\CCCCCCCC(=O)O,75.0,,REACTANT:CCCC[Sn](CCCC)(CCCC)c1cccs1.CCCCc1nc(...
2,,CC(C)CCBr.[Li]c1cccs1,,O,,,CCc1c(Cc2[nH]c(C(=O)OCc3ccccc3)c(C)c2CCC(=O)OC...,88.0,0.0,REACTANT:CC(C)CCBr.[Li]c1cccs1PRODUCT:CCc1c(Cc...
3,,CS(=O)(=O)O.O=P12OP3(=O)OP(=O)(O1)OP(=O)(O2)O3...,,,,,Oc1cc2cc[nH]c2cc1O,95.0,90.0,REACTANT:CS(=O)(=O)O.O=P12OP3(=O)OP(=O)(O1)OP(...
4,,C=C[Mg]Br.CC(C)CC=O.CCOC(=O)CC(=O)OCC.[K+].[OH-],,C1CCOC1.CCO,,,NN=C(C=Cc1ccccc1)c1ccccc1,30.0,0.0,REACTANT:C=C[Mg]Br.CC(C)CC=O.CCOC(=O)CC(=O)OCC...
...,...,...,...,...,...,...,...,...,...,...
545630,,Nc1cccc2c1C(=O)N(C1CCC(=O)NC1=O)C2=O.O=C(Cl)c1...,,C1CCOC1,,,CCCCCCCC/C=C\CCCCCCCC(=O)O,88.0,,REACTANT:Nc1cccc2c1C(=O)N(C1CCC(=O)NC1=O)C2=O....
545631,,CC(=O)O[BH-](OC(C)=O)OC(C)=O.O=C(CNC(=O)c1cccc...,,CC(=O)O.CCOC(C)=O.ClCCl,,,O=C(O)c1ccccc1-c1c2ccc(=O)cc-2oc2cc(O)ccc12,90.0,,REACTANT:CC(=O)O[BH-](OC(C)=O)OC(C)=O.O=C(CNC(...
545632,,CN1CCOCC1.COC(=O)c1ccc(Cc2cn(C)c3ccc(N)cc23)c(...,,ClCCl,,,CC#CN1C(=O)C(C)Oc2ccc(-n3c(=O)cc(C(F)(F)F)[nH]...,74.0,,REACTANT:CN1CCOCC1.COC(=O)c1ccc(Cc2cn(C)c3ccc(...
545633,,Cc1ccc(N)c(C#C[Si](C)(C)C)n1.[Na+].[OH-],,CO,,,CC(C)CN=C1C(c2ccccc2)=C(c2ccccc2)C(c2ccccc2)=C...,75.0,0.0,REACTANT:Cc1ccc(N)c(C#C[Si](C)(C)C)n1.[Na+].[O...


In [3]:
train_ds = train_ds[['input', 'YIELD']]
train_ds

,input,YIELD
0,REACTANT:CC(=O)Cl.CC(C)(C)OC(=O)N1CCC2(CC1)CC(...,96.0
1,REACTANT:CCCC[Sn](CCCC)(CCCC)c1cccs1.CCCCc1nc(...,75.0
2,REACTANT:CC(C)CCBr.[Li]c1cccs1PRODUCT:CCc1c(Cc...,88.0
3,REACTANT:CS(=O)(=O)O.O=P12OP3(=O)OP(=O)(O1)OP(...,95.0
4,REACTANT:C=C[Mg]Br.CC(C)CC=O.CCOC(=O)CC(=O)OCC...,30.0
...,...,...
545630,REACTANT:Nc1cccc2c1C(=O)N(C1CCC(=O)NC1=O)C2=O....,88.0
545631,REACTANT:CC(=O)O[BH-](OC(C)=O)OC(C)=O.O=C(CNC(...,90.0
545632,REACTANT:CN1CCOCC1.COC(=O)c1ccc(Cc2cn(C)c3ccc(...,74.0
545633,REACTANT:Cc1ccc(N)c(C#C[Si](C)(C)C)n1.[Na+].[O...,75.0


In [55]:
ori = pd.read_csv('../../all_ord_reaction_uniq_with_attr_v3.tsv').drop_duplicates().reset_index(drop=True)
ori

,CATALYST,REACTANT,REAGENT,SOLVENT,INTERNAL_STANDARD,NoData,PRODUCT,YIELD,TEMP
0,CC(C)(C)c1ccn2->[Ir+]34(<-n5cc(C(F)(F)F)ccc5-c...,CC(=O)c1ccc(Br)cc1.O=C(O)C1CCCN1C(=O)OCc1ccccc1,O=C([O-])[O-].[Cs+],CN(C)C=O,NaN,NaN,CC(=O)c1ccc(C2CCCN2C(=O)OCc2ccccc2)cc1,93.0,23.0
1,CC(C)(C)c1ccn2->[Ir+]34(<-n5cc(C(F)(F)F)ccc5-c...,CC(=O)c1ccc(Br)cc1.CC(C)(C)OC(=O)N1CCCCC1C(=O)O,O=C([O-])[O-].[Cs+],CN(C)C=O,NaN,NaN,CC(=O)c1ccc(C2CCCN2C(=O)OCc2ccccc2)cc1,82.0,23.0
2,CC(C)(C)c1ccn2->[Ir+]34(<-n5cc(C(F)(F)F)ccc5-c...,CC(=O)c1ccc(Br)cc1.CC(C)(C)OC(=O)N1CCOCC1C(=O)O,O=C([O-])[O-].[Cs+],CN(C)C=O,NaN,NaN,CC(=O)c1ccc(C2CCCN2C(=O)OCc2ccccc2)cc1,61.0,23.0
3,CC(C)(C)c1ccn2->[Ir+]34(<-n5cc(C(F)(F)F)ccc5-c...,CC(=O)c1ccc(Br)cc1.CC(C)C(NC(=O)OC(C)(C)C)C(=O)O,O=C([O-])[O-].[Cs+],CN(C)C=O,NaN,NaN,CC(=O)c1ccc(C2CCCN2C(=O)OCc2ccccc2)cc1,72.0,23.0
4,CC(C)(C)c1ccn2->[Ir+]34(<-n5cc(C(F)(F)F)ccc5-c...,CC(=O)c1ccc(Br)cc1.CC(C)(C)OC(=O)NC(Cc1cn(C(=O...,O=C([O-])[O-].[Cs+],CN(C)C=O,NaN,NaN,CC(=O)c1ccc(C2CCCN2C(=O)OCc2ccccc2)cc1,83.0,23.0
...,...,...,...,...,...,...,...,...,...
2058485,NaN,COc1ccccc1CCCCBr.Fc1ccc2c(C3CCNCC3)noc2c1.O=C(...,NaN,CC#N.CCO,NaN,NaN,CC(O)CC(=O)[O-].O=C([O-])CCCO,NaN,NaN
2058486,NaN,CC(O)=S.CCOC(=O)N=NC(=O)OCC.O=C1C[C@@H](O)CN1....,NaN,C1CCOC1,NaN,NaN,CC(O)CC(=O)[O-].O=C([O-])CCCO,NaN,NaN
2058487,NaN,C[C@@H](O[Si](C)(C)C(C)(C)C)[C@H]1C(=O)N2C(C(=...,NaN,CC#N,NaN,NaN,CC(O)CC(=O)[O-].O=C([O-])CCCO,NaN,NaN
2058488,CC(=O)[O-].O.[Cu+2],C=O.CC(=O)O.CC(=O)[O-].CCOC(=O)CC(=O)OCC.[K+],NaN,NaN,NaN,NaN,O=S(=O)(c1ccc(Cl)cc1)C(F)(F)F,NaN,NaN


In [59]:
len(ori['REACTANT'].unique()), len(ori['PRODUCT'].unique())

(1045802, 440212)

In [60]:
df = ori[~ori['YIELD'].isna()]
df['YIELD'] = df['YIELD'].clip(0, 100)
df

,CATALYST,REACTANT,REAGENT,SOLVENT,INTERNAL_STANDARD,NoData,PRODUCT,YIELD,TEMP
0,CC(C)(C)c1ccn2->[Ir+]34(<-n5cc(C(F)(F)F)ccc5-c...,CC(=O)c1ccc(Br)cc1.O=C(O)C1CCCN1C(=O)OCc1ccccc1,O=C([O-])[O-].[Cs+],CN(C)C=O,NaN,NaN,CC(=O)c1ccc(C2CCCN2C(=O)OCc2ccccc2)cc1,93.0,23.0
1,CC(C)(C)c1ccn2->[Ir+]34(<-n5cc(C(F)(F)F)ccc5-c...,CC(=O)c1ccc(Br)cc1.CC(C)(C)OC(=O)N1CCCCC1C(=O)O,O=C([O-])[O-].[Cs+],CN(C)C=O,NaN,NaN,CC(=O)c1ccc(C2CCCN2C(=O)OCc2ccccc2)cc1,82.0,23.0
2,CC(C)(C)c1ccn2->[Ir+]34(<-n5cc(C(F)(F)F)ccc5-c...,CC(=O)c1ccc(Br)cc1.CC(C)(C)OC(=O)N1CCOCC1C(=O)O,O=C([O-])[O-].[Cs+],CN(C)C=O,NaN,NaN,CC(=O)c1ccc(C2CCCN2C(=O)OCc2ccccc2)cc1,61.0,23.0
3,CC(C)(C)c1ccn2->[Ir+]34(<-n5cc(C(F)(F)F)ccc5-c...,CC(=O)c1ccc(Br)cc1.CC(C)C(NC(=O)OC(C)(C)C)C(=O)O,O=C([O-])[O-].[Cs+],CN(C)C=O,NaN,NaN,CC(=O)c1ccc(C2CCCN2C(=O)OCc2ccccc2)cc1,72.0,23.0
4,CC(C)(C)c1ccn2->[Ir+]34(<-n5cc(C(F)(F)F)ccc5-c...,CC(=O)c1ccc(Br)cc1.CC(C)(C)OC(=O)NC(Cc1cn(C(=O...,O=C([O-])[O-].[Cs+],CN(C)C=O,NaN,NaN,CC(=O)c1ccc(C2CCCN2C(=O)OCc2ccccc2)cc1,83.0,23.0
...,...,...,...,...,...,...,...,...,...
2058408,[Na+].[OH-],COc1ccc(CC#N)cc1OC.O=Cc1ccc2ccccc2c1,NaN,CCO,NaN,NaN,O=C(F)OCC(F)(F)F,82.0,NaN
2058469,NaN,CNCC[C@@H](O)c1ccccc1.FC(F)(F)c1ccc(Cl)cc1.[H-...,NaN,CC(=O)N(C)C,NaN,NaN,CC(O)CC(=O)[O-].O=C([O-])CCCO,86.0,90.0
2058470,NaN,COc1cc(OC)c(Br)c(OC)c1.O.O=C1CCCCC1,NaN,C1CCOC1,NaN,NaN,CC(O)CC(=O)[O-].O=C([O-])CCCO,65.0,-30.0
2058473,NaN,CN(C)Cc1ccccc1.OCC1CO1.OCCS,NaN,CC(=O)CC(C)C,NaN,NaN,CC(O)CC(=O)[O-].O=C([O-])CCCO,90.0,50.0


In [62]:
len(df['REACTANT'].unique()), len(df['PRODUCT'].unique())

(399405, 358)

In [44]:
df['REACTANT'].isna().sum(), df['PRODUCT'].isna().sum(), (df['REACTANT'].isna() & df['PRODUCT'].isna()).sum()

(144, 142, 142)

In [63]:
df = df[~(df['REACTANT'].isna() | df['PRODUCT'].isna())]

,CATALYST,REACTANT,REAGENT,SOLVENT,INTERNAL_STANDARD,NoData,PRODUCT,YIELD,TEMP
0,CC(C)(C)c1ccn2->[Ir+]34(<-n5cc(C(F)(F)F)ccc5-c...,CC(=O)c1ccc(Br)cc1.O=C(O)C1CCCN1C(=O)OCc1ccccc1,O=C([O-])[O-].[Cs+],CN(C)C=O,NaN,NaN,CC(=O)c1ccc(C2CCCN2C(=O)OCc2ccccc2)cc1,93.0,23.0
1,CC(C)(C)c1ccn2->[Ir+]34(<-n5cc(C(F)(F)F)ccc5-c...,CC(=O)c1ccc(Br)cc1.CC(C)(C)OC(=O)N1CCCCC1C(=O)O,O=C([O-])[O-].[Cs+],CN(C)C=O,NaN,NaN,CC(=O)c1ccc(C2CCCN2C(=O)OCc2ccccc2)cc1,82.0,23.0
2,CC(C)(C)c1ccn2->[Ir+]34(<-n5cc(C(F)(F)F)ccc5-c...,CC(=O)c1ccc(Br)cc1.CC(C)(C)OC(=O)N1CCOCC1C(=O)O,O=C([O-])[O-].[Cs+],CN(C)C=O,NaN,NaN,CC(=O)c1ccc(C2CCCN2C(=O)OCc2ccccc2)cc1,61.0,23.0
3,CC(C)(C)c1ccn2->[Ir+]34(<-n5cc(C(F)(F)F)ccc5-c...,CC(=O)c1ccc(Br)cc1.CC(C)C(NC(=O)OC(C)(C)C)C(=O)O,O=C([O-])[O-].[Cs+],CN(C)C=O,NaN,NaN,CC(=O)c1ccc(C2CCCN2C(=O)OCc2ccccc2)cc1,72.0,23.0
4,CC(C)(C)c1ccn2->[Ir+]34(<-n5cc(C(F)(F)F)ccc5-c...,CC(=O)c1ccc(Br)cc1.CC(C)(C)OC(=O)NC(Cc1cn(C(=O...,O=C([O-])[O-].[Cs+],CN(C)C=O,NaN,NaN,CC(=O)c1ccc(C2CCCN2C(=O)OCc2ccccc2)cc1,83.0,23.0
...,...,...,...,...,...,...,...,...,...
2058408,[Na+].[OH-],COc1ccc(CC#N)cc1OC.O=Cc1ccc2ccccc2c1,NaN,CCO,NaN,NaN,O=C(F)OCC(F)(F)F,82.0,NaN
2058469,NaN,CNCC[C@@H](O)c1ccccc1.FC(F)(F)c1ccc(Cl)cc1.[H-...,NaN,CC(=O)N(C)C,NaN,NaN,CC(O)CC(=O)[O-].O=C([O-])CCCO,86.0,90.0
2058470,NaN,COc1cc(OC)c(Br)c(OC)c1.O.O=C1CCCCC1,NaN,C1CCOC1,NaN,NaN,CC(O)CC(=O)[O-].O=C([O-])CCCO,65.0,-30.0
2058473,NaN,CN(C)Cc1ccccc1.OCC1CO1.OCCS,NaN,CC(=O)CC(C)C,NaN,NaN,CC(O)CC(=O)[O-].O=C([O-])CCCO,90.0,50.0


In [64]:
df['input'] = 'REACTANT:' + df['REACTANT'] + 'PRODUCT:' + df['PRODUCT']
df = df[['input', 'YIELD']].drop_duplicates().reset_index(drop=True)
df

,input,YIELD
0,REACTANT:CC(=O)c1ccc(Br)cc1.O=C(O)C1CCCN1C(=O)...,93.0
1,REACTANT:CC(=O)c1ccc(Br)cc1.CC(C)(C)OC(=O)N1CC...,82.0
2,REACTANT:CC(=O)c1ccc(Br)cc1.CC(C)(C)OC(=O)N1CC...,61.0
3,REACTANT:CC(=O)c1ccc(Br)cc1.CC(C)C(NC(=O)OC(C)...,72.0
4,REACTANT:CC(=O)c1ccc(Br)cc1.CC(C)(C)OC(=O)NC(C...,83.0
...,...,...
598384,REACTANT:COc1ccc(CC#N)cc1OC.O=Cc1ccc2ccccc2c1P...,82.0
598385,REACTANT:CNCC[C@@H](O)c1ccccc1.FC(F)(F)c1ccc(C...,86.0
598386,REACTANT:COc1cc(OC)c(Br)c(OC)c1.O.O=C1CCCCC1PR...,65.0
598387,REACTANT:CN(C)Cc1ccccc1.OCC1CO1.OCCSPRODUCT:CC...,90.0


In [51]:
len(df['input'].unique())

589667

In [54]:
df[df['input']=='REACTANT:Cn1cnc(C#N)c1.Fc1ccccc1BrPRODUCT:Cn1cnc(C#N)c1-c1ccccc1F.Cn1cnc(C#N)c1-c1ccccc1F']

,input,YIELD
137311,REACTANT:Cn1cnc(C#N)c1.Fc1ccccc1BrPRODUCT:Cn1c...,29.0
137312,REACTANT:Cn1cnc(C#N)c1.Fc1ccccc1BrPRODUCT:Cn1c...,45.0
137313,REACTANT:Cn1cnc(C#N)c1.Fc1ccccc1BrPRODUCT:Cn1c...,50.0
137314,REACTANT:Cn1cnc(C#N)c1.Fc1ccccc1BrPRODUCT:Cn1c...,6.0
137315,REACTANT:Cn1cnc(C#N)c1.Fc1ccccc1BrPRODUCT:Cn1c...,4.0
...,...,...
312939,REACTANT:Cn1cnc(C#N)c1.Fc1ccccc1BrPRODUCT:Cn1c...,92.0
312940,REACTANT:Cn1cnc(C#N)c1.Fc1ccccc1BrPRODUCT:Cn1c...,98.0
312941,REACTANT:Cn1cnc(C#N)c1.Fc1ccccc1BrPRODUCT:Cn1c...,100.0
312942,REACTANT:Cn1cnc(C#N)c1.Fc1ccccc1BrPRODUCT:Cn1c...,96.0


In [67]:
dfagg = df.groupby('input')['YIELD'].agg('mean').reset_index()
dfagg

,input,YIELD
0,REACTANT:*C(F)(F)C(*)(F)F.C1=CCCC1.Cl[SiH](Cl)...,91.0
1,REACTANT:*C(F)(F)C(*)(F)F.C1CCNCC1.CC(C)(C)[O-...,77.0
2,REACTANT:*C(F)(F)C(*)(F)F.CC(=O)c1ccccc1.[BH4-...,95.0
3,REACTANT:*C(F)(F)C(*)(F)F.CC(=O)c1ccccc1.[H][H...,95.0
4,REACTANT:*C(F)(F)C(*)(F)F.CCCCC(N)N.CCOCC.Cc1c...,21.0
...,...,...
589661,REACTANT:c1cncc(C2CCCC2)c1PRODUCT:C[C@H](CO)CO...,84.0
589662,REACTANT:c1cncc(OCCOC2CCCCO2)c1PRODUCT:CC(C)(C...,86.0
589663,REACTANT:c1cncc(OCCOC2CCCCO2)c1PRODUCT:CC1CO1,86.0
589664,REACTANT:c1cncc(OCCOC2CCCCO2)c1PRODUCT:COC(=O)...,86.0


In [78]:
lens = dfagg['input'].apply(lambda x: len(x))
# remove data that have too long inputs
dfagg = dfagg[lens <= 512].reset_index(drop=True)
dfagg

,input,YIELD
0,REACTANT:*C(F)(F)C(*)(F)F.C1=CCCC1.Cl[SiH](Cl)...,91.0
1,REACTANT:*C(F)(F)C(*)(F)F.C1CCNCC1.CC(C)(C)[O-...,77.0
2,REACTANT:*C(F)(F)C(*)(F)F.CC(=O)c1ccccc1.[BH4-...,95.0
3,REACTANT:*C(F)(F)C(*)(F)F.CC(=O)c1ccccc1.[H][H...,95.0
4,REACTANT:*C(F)(F)C(*)(F)F.CCCCC(N)N.CCOCC.Cc1c...,21.0
...,...,...
589651,REACTANT:c1cncc(C2CCCC2)c1PRODUCT:C[C@H](CO)CO...,84.0
589652,REACTANT:c1cncc(OCCOC2CCCCO2)c1PRODUCT:CC(C)(C...,86.0
589653,REACTANT:c1cncc(OCCOC2CCCCO2)c1PRODUCT:CC1CO1,86.0
589654,REACTANT:c1cncc(OCCOC2CCCCO2)c1PRODUCT:COC(=O)...,86.0


In [83]:
# multiinput
ori = pd.read_csv('../../all_ord_reaction_uniq_with_attr_v3.tsv').drop_duplicates().reset_index(drop=True)
ori

,CATALYST,REACTANT,REAGENT,SOLVENT,INTERNAL_STANDARD,NoData,PRODUCT,YIELD,TEMP
0,CC(C)(C)c1ccn2->[Ir+]34(<-n5cc(C(F)(F)F)ccc5-c...,CC(=O)c1ccc(Br)cc1.O=C(O)C1CCCN1C(=O)OCc1ccccc1,O=C([O-])[O-].[Cs+],CN(C)C=O,NaN,NaN,CC(=O)c1ccc(C2CCCN2C(=O)OCc2ccccc2)cc1,93.0,23.0
1,CC(C)(C)c1ccn2->[Ir+]34(<-n5cc(C(F)(F)F)ccc5-c...,CC(=O)c1ccc(Br)cc1.CC(C)(C)OC(=O)N1CCCCC1C(=O)O,O=C([O-])[O-].[Cs+],CN(C)C=O,NaN,NaN,CC(=O)c1ccc(C2CCCN2C(=O)OCc2ccccc2)cc1,82.0,23.0
2,CC(C)(C)c1ccn2->[Ir+]34(<-n5cc(C(F)(F)F)ccc5-c...,CC(=O)c1ccc(Br)cc1.CC(C)(C)OC(=O)N1CCOCC1C(=O)O,O=C([O-])[O-].[Cs+],CN(C)C=O,NaN,NaN,CC(=O)c1ccc(C2CCCN2C(=O)OCc2ccccc2)cc1,61.0,23.0
3,CC(C)(C)c1ccn2->[Ir+]34(<-n5cc(C(F)(F)F)ccc5-c...,CC(=O)c1ccc(Br)cc1.CC(C)C(NC(=O)OC(C)(C)C)C(=O)O,O=C([O-])[O-].[Cs+],CN(C)C=O,NaN,NaN,CC(=O)c1ccc(C2CCCN2C(=O)OCc2ccccc2)cc1,72.0,23.0
4,CC(C)(C)c1ccn2->[Ir+]34(<-n5cc(C(F)(F)F)ccc5-c...,CC(=O)c1ccc(Br)cc1.CC(C)(C)OC(=O)NC(Cc1cn(C(=O...,O=C([O-])[O-].[Cs+],CN(C)C=O,NaN,NaN,CC(=O)c1ccc(C2CCCN2C(=O)OCc2ccccc2)cc1,83.0,23.0
...,...,...,...,...,...,...,...,...,...
2058485,NaN,COc1ccccc1CCCCBr.Fc1ccc2c(C3CCNCC3)noc2c1.O=C(...,NaN,CC#N.CCO,NaN,NaN,CC(O)CC(=O)[O-].O=C([O-])CCCO,NaN,NaN
2058486,NaN,CC(O)=S.CCOC(=O)N=NC(=O)OCC.O=C1C[C@@H](O)CN1....,NaN,C1CCOC1,NaN,NaN,CC(O)CC(=O)[O-].O=C([O-])CCCO,NaN,NaN
2058487,NaN,C[C@@H](O[Si](C)(C)C(C)(C)C)[C@H]1C(=O)N2C(C(=...,NaN,CC#N,NaN,NaN,CC(O)CC(=O)[O-].O=C([O-])CCCO,NaN,NaN
2058488,CC(=O)[O-].O.[Cu+2],C=O.CC(=O)O.CC(=O)[O-].CCOC(=O)CC(=O)OCC.[K+],NaN,NaN,NaN,NaN,O=S(=O)(c1ccc(Cl)cc1)C(F)(F)F,NaN,NaN


In [84]:
df = ori[~ori['PRODUCT'].isna()]
df

,CATALYST,REACTANT,REAGENT,SOLVENT,INTERNAL_STANDARD,NoData,PRODUCT,YIELD,TEMP
0,CC(C)(C)c1ccn2->[Ir+]34(<-n5cc(C(F)(F)F)ccc5-c...,CC(=O)c1ccc(Br)cc1.O=C(O)C1CCCN1C(=O)OCc1ccccc1,O=C([O-])[O-].[Cs+],CN(C)C=O,NaN,NaN,CC(=O)c1ccc(C2CCCN2C(=O)OCc2ccccc2)cc1,93.0,23.0
1,CC(C)(C)c1ccn2->[Ir+]34(<-n5cc(C(F)(F)F)ccc5-c...,CC(=O)c1ccc(Br)cc1.CC(C)(C)OC(=O)N1CCCCC1C(=O)O,O=C([O-])[O-].[Cs+],CN(C)C=O,NaN,NaN,CC(=O)c1ccc(C2CCCN2C(=O)OCc2ccccc2)cc1,82.0,23.0
2,CC(C)(C)c1ccn2->[Ir+]34(<-n5cc(C(F)(F)F)ccc5-c...,CC(=O)c1ccc(Br)cc1.CC(C)(C)OC(=O)N1CCOCC1C(=O)O,O=C([O-])[O-].[Cs+],CN(C)C=O,NaN,NaN,CC(=O)c1ccc(C2CCCN2C(=O)OCc2ccccc2)cc1,61.0,23.0
3,CC(C)(C)c1ccn2->[Ir+]34(<-n5cc(C(F)(F)F)ccc5-c...,CC(=O)c1ccc(Br)cc1.CC(C)C(NC(=O)OC(C)(C)C)C(=O)O,O=C([O-])[O-].[Cs+],CN(C)C=O,NaN,NaN,CC(=O)c1ccc(C2CCCN2C(=O)OCc2ccccc2)cc1,72.0,23.0
4,CC(C)(C)c1ccn2->[Ir+]34(<-n5cc(C(F)(F)F)ccc5-c...,CC(=O)c1ccc(Br)cc1.CC(C)(C)OC(=O)NC(Cc1cn(C(=O...,O=C([O-])[O-].[Cs+],CN(C)C=O,NaN,NaN,CC(=O)c1ccc(C2CCCN2C(=O)OCc2ccccc2)cc1,83.0,23.0
...,...,...,...,...,...,...,...,...,...
2058485,NaN,COc1ccccc1CCCCBr.Fc1ccc2c(C3CCNCC3)noc2c1.O=C(...,NaN,CC#N.CCO,NaN,NaN,CC(O)CC(=O)[O-].O=C([O-])CCCO,NaN,NaN
2058486,NaN,CC(O)=S.CCOC(=O)N=NC(=O)OCC.O=C1C[C@@H](O)CN1....,NaN,C1CCOC1,NaN,NaN,CC(O)CC(=O)[O-].O=C([O-])CCCO,NaN,NaN
2058487,NaN,C[C@@H](O[Si](C)(C)C(C)(C)C)[C@H]1C(=O)N2C(C(=...,NaN,CC#N,NaN,NaN,CC(O)CC(=O)[O-].O=C([O-])CCCO,NaN,NaN
2058488,CC(=O)[O-].O.[Cu+2],C=O.CC(=O)O.CC(=O)[O-].CCOC(=O)CC(=O)OCC.[K+],NaN,NaN,NaN,NaN,O=S(=O)(c1ccc(Cl)cc1)C(F)(F)F,NaN,NaN


In [88]:
len(df['REACTANT'].unique()), len(df['PRODUCT'].unique())

(1045802, 440211)

In [94]:
dfr = df[df['REACTANT'].isna()]
dfr

,CATALYST,REACTANT,REAGENT,SOLVENT,INTERNAL_STANDARD,NoData,PRODUCT,YIELD,TEMP
583,NaN,NaN,CC(=O)OP(=O)([O-])[O-].Nc1ncnc2c1ncn2[C@@H]1O[...,O,NaN,NaN,C#C[C@]1(CO)O[C@@H](n2cnc3c(N)nc(F)nc32)C[C@@H]1O,NaN,NaN
584,NaN,NaN,CC=O.Cl[Mn]Cl.Nc1[nH]c(F)nc2ncnc1-2.O.OCCN(CCO...,CC(C)O.O,NaN,NaN,C#C[C@](O)(C=O)CO,76.0,35.0
66975,NaN,NaN,NaN,NaN,NaN,C1CCOC1.CC(C)C[Mg+].CON(C)C(=O)c1ccc(O)nc1.[Cl-],CC(C)CC(=O)c1ccc(O)nc1,NaN,NaN
66976,NaN,NaN,NaN,NaN,NaN,CN.O.O=C(O)c1ccc(Cl)c([N+](=O)[O-])c1,CNc1ccc(C(=O)O)cc1[N+](=O)[O-],NaN,NaN
66977,NaN,NaN,NaN,NaN,NaN,CCn1cc(C(=O)O)c(=O)c2cc(F)c(-c3ccc(N)cc3)cc21....,CCn1cc(C(=O)O)c(=O)c2cc(F)c(-c3ccc(NC=O)cc3)cc21,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1869304,NaN,NaN,NaN,NaN,NaN,CCN1CC2OC2C1.CCOc1ccccc1O.Cl.ClCCl,CCOc1ccccc1OC1CN(CC)CC1O,NaN,NaN
1869305,NaN,NaN,NaN,NaN,NaN,CC(=O)O.CCOC(=O)C(=Cc1cccc(OCCc2ccc(OS(C)(=O)=...,CCOC(=O)C(Cc1cccc(OCCc2ccc(OS(C)(=O)=O)cc2)c1)OCC,NaN,NaN
1869306,NaN,NaN,NaN,NaN,NaN,CCCCC1CCNCC1.Cc1ccc(S(=O)(=O)OCC(C)Cn2c(=O)sc3...,CCCCC1CCN(CC(C)Cn2c(=O)sc3ccccc32)CC1,NaN,NaN
1869307,NaN,NaN,NaN,NaN,NaN,CC(C)I.CN(C)C=O.COC(=O)c1cc(Br)cc2[nH]ccc12.O....,COC(=O)c1cc(Br)cc2c1ccn2C(C)C,NaN,NaN


In [95]:
len(dfr['REACTANT'].unique()), len(dfr['PRODUCT'].unique())

(1, 439901)

In [2]:
df = pd.read_csv(CFG.data_path).drop_duplicates().reset_index(drop=True)
df = df[~df['YIELD'].isna()].reset_index(drop=True)
df['YIELD'] = df['YIELD'].clip(0, 100)/100
df = df[~(df['REACTANT'].isna() | df['PRODUCT'].isna())]
for col in ['CATALYST', 'REACTANT', 'REAGENT', 'SOLVENT', 'INTERNAL_STANDARD', 'NoData','PRODUCT']:
    df[col] = df[col].fillna(' ')
df

,CATALYST,REACTANT,REAGENT,SOLVENT,INTERNAL_STANDARD,NoData,PRODUCT,YIELD,TEMP
0,CC(C)(C)c1ccn2->[Ir+]34(<-n5cc(C(F)(F)F)ccc5-c...,CC(=O)c1ccc(Br)cc1.O=C(O)C1CCCN1C(=O)OCc1ccccc1,O=C([O-])[O-].[Cs+],CN(C)C=O,,,CC(=O)c1ccc(C2CCCN2C(=O)OCc2ccccc2)cc1,0.93,23.0
1,CC(C)(C)c1ccn2->[Ir+]34(<-n5cc(C(F)(F)F)ccc5-c...,CC(=O)c1ccc(Br)cc1.CC(C)(C)OC(=O)N1CCCCC1C(=O)O,O=C([O-])[O-].[Cs+],CN(C)C=O,,,CC(=O)c1ccc(C2CCCN2C(=O)OCc2ccccc2)cc1,0.82,23.0
2,CC(C)(C)c1ccn2->[Ir+]34(<-n5cc(C(F)(F)F)ccc5-c...,CC(=O)c1ccc(Br)cc1.CC(C)(C)OC(=O)N1CCOCC1C(=O)O,O=C([O-])[O-].[Cs+],CN(C)C=O,,,CC(=O)c1ccc(C2CCCN2C(=O)OCc2ccccc2)cc1,0.61,23.0
3,CC(C)(C)c1ccn2->[Ir+]34(<-n5cc(C(F)(F)F)ccc5-c...,CC(=O)c1ccc(Br)cc1.CC(C)C(NC(=O)OC(C)(C)C)C(=O)O,O=C([O-])[O-].[Cs+],CN(C)C=O,,,CC(=O)c1ccc(C2CCCN2C(=O)OCc2ccccc2)cc1,0.72,23.0
4,CC(C)(C)c1ccn2->[Ir+]34(<-n5cc(C(F)(F)F)ccc5-c...,CC(=O)c1ccc(Br)cc1.CC(C)(C)OC(=O)NC(Cc1cn(C(=O...,O=C([O-])[O-].[Cs+],CN(C)C=O,,,CC(=O)c1ccc(C2CCCN2C(=O)OCc2ccccc2)cc1,0.83,23.0
...,...,...,...,...,...,...,...,...,...
609637,[Na+].[OH-],COc1ccc(CC#N)cc1OC.O=Cc1ccc2ccccc2c1,,CCO,,,O=C(F)OCC(F)(F)F,0.82,NaN
609638,,CNCC[C@@H](O)c1ccccc1.FC(F)(F)c1ccc(Cl)cc1.[H-...,,CC(=O)N(C)C,,,CC(O)CC(=O)[O-].O=C([O-])CCCO,0.86,90.0
609639,,COc1cc(OC)c(Br)c(OC)c1.O.O=C1CCCCC1,,C1CCOC1,,,CC(O)CC(=O)[O-].O=C([O-])CCCO,0.65,-30.0
609640,,CN(C)Cc1ccccc1.OCC1CO1.OCCS,,CC(=O)CC(C)C,,,CC(O)CC(=O)[O-].O=C([O-])CCCO,0.90,50.0


In [7]:
def clean(row):
    row = row.replace('. ', '').replace(' .', '').replace('  ', ' ')
    return row
df['REAGENT'] = df['CATALYST'] + '.' + df['REAGENT']
df

In [8]:
for idx, row in df.iterrows():
    print(row['REAGENT'])
    if '. ' in row['REAGENT']:
        print(idx)
        break

CC(C)(C)c1ccn2->[Ir+]34(<-n5cc(C(F)(F)F)ccc5-c5c(F)cc(F)cc53)(<-n3cc(C(F)(F)F)ccc3-c3c(F)cc(F)cc34)<-n3ccc(C(C)(C)C)cc3-c2c1.CC(C)(C)c1ccnc(-c2cc(C(C)(C)C)ccn2)c1.COCCOC.Cl[Ni]Cl.F[P-](F)(F)(F)(F)F.O=C([O-])[O-].[Cs+]
CC(C)(C)c1ccn2->[Ir+]34(<-n5cc(C(F)(F)F)ccc5-c5c(F)cc(F)cc53)(<-n3cc(C(F)(F)F)ccc3-c3c(F)cc(F)cc34)<-n3ccc(C(C)(C)C)cc3-c2c1.CC(C)(C)c1ccnc(-c2cc(C(C)(C)C)ccn2)c1.COCCOC.Cl[Ni]Cl.F[P-](F)(F)(F)(F)F.O=C([O-])[O-].[Cs+]
CC(C)(C)c1ccn2->[Ir+]34(<-n5cc(C(F)(F)F)ccc5-c5c(F)cc(F)cc53)(<-n3cc(C(F)(F)F)ccc3-c3c(F)cc(F)cc34)<-n3ccc(C(C)(C)C)cc3-c2c1.CC(C)(C)c1ccnc(-c2cc(C(C)(C)C)ccn2)c1.COCCOC.Cl[Ni]Cl.F[P-](F)(F)(F)(F)F.O=C([O-])[O-].[Cs+]
CC(C)(C)c1ccn2->[Ir+]34(<-n5cc(C(F)(F)F)ccc5-c5c(F)cc(F)cc53)(<-n3cc(C(F)(F)F)ccc3-c3c(F)cc(F)cc34)<-n3ccc(C(C)(C)C)cc3-c2c1.CC(C)(C)c1ccnc(-c2cc(C(C)(C)C)ccn2)c1.COCCOC.Cl[Ni]Cl.F[P-](F)(F)(F)(F)F.O=C([O-])[O-].[Cs+]
CC(C)(C)c1ccn2->[Ir+]34(<-n5cc(C(F)(F)F)ccc5-c5c(F)cc(F)cc53)(<-n3cc(C(F)(F)F)ccc3-c3c(F)cc(F)cc34)<-n3ccc(C(C)(C)C)cc3-c2c1.CC(

In [11]:
df['REAGENT'] = df['REAGENT'].apply(lambda x: clean(x))
# for idx, row in df.iterrows():
#     print(row['REAGENT'])
# #     if '. ' in row['REAGENT']:
# #         print(idx)
# #         break
#     if idx == 104:
#         break

In [13]:
from rdkit import Chem
def canonicalize(mol):
    mol = Chem.MolToSmiles(Chem.MolFromSmiles(mol),True)
    return mol

df['REAGENT'] = df['REAGENT'].apply(lambda x: canonicalize(x) if x != ' ' else ' ')
for idx, row in df.iterrows():
    print(row['REAGENT'])

    if idx == 104:
        break

[12:45:14] WARNING: not removing hydrogen atom without neighbors
[12:45:14] WARNING: not removing hydrogen atom without neighbors
[12:45:14] WARNING: not removing hydrogen atom without neighbors
[12:45:14] WARNING: not removing hydrogen atom without neighbors
[12:45:14] WARNING: not removing hydrogen atom without neighbors
[12:45:14] WARNING: not removing hydrogen atom without neighbors
[12:45:15] WARNING: not removing hydrogen atom without neighbors
[12:45:15] WARNING: not removing hydrogen atom without neighbors
[12:45:16] WARNING: not removing hydrogen atom without neighbors
[12:45:17] WARNING: not removing hydrogen atom without neighbors
[12:45:18] WARNING: not removing hydrogen atom without neighbors
[12:45:18] WARNING: not removing hydrogen atom without neighbors
[12:45:18] WARNING: not removing hydrogen atom without neighbors
[12:45:18] WARNING: not removing hydrogen atom without neighbors
[12:45:18] WARNING: not removing hydrogen atom without neighbors
[12:45:19] WARNING: not r

CC(C)(C)c1ccn2->[Ir+]34(<-n5cc(C(F)(F)F)ccc5-c5c(F)cc(F)cc53)(<-n3cc(C(F)(F)F)ccc3-c3c(F)cc(F)cc34)<-n3ccc(C(C)(C)C)cc3-c2c1.CC(C)(C)c1ccnc(-c2cc(C(C)(C)C)ccn2)c1.COCCOC.Cl[Ni]Cl.F[P-](F)(F)(F)(F)F.O=C([O-])[O-].[Cs+]
CC(C)(C)c1ccn2->[Ir+]34(<-n5cc(C(F)(F)F)ccc5-c5c(F)cc(F)cc53)(<-n3cc(C(F)(F)F)ccc3-c3c(F)cc(F)cc34)<-n3ccc(C(C)(C)C)cc3-c2c1.CC(C)(C)c1ccnc(-c2cc(C(C)(C)C)ccn2)c1.COCCOC.Cl[Ni]Cl.F[P-](F)(F)(F)(F)F.O=C([O-])[O-].[Cs+]
CC(C)(C)c1ccn2->[Ir+]34(<-n5cc(C(F)(F)F)ccc5-c5c(F)cc(F)cc53)(<-n3cc(C(F)(F)F)ccc3-c3c(F)cc(F)cc34)<-n3ccc(C(C)(C)C)cc3-c2c1.CC(C)(C)c1ccnc(-c2cc(C(C)(C)C)ccn2)c1.COCCOC.Cl[Ni]Cl.F[P-](F)(F)(F)(F)F.O=C([O-])[O-].[Cs+]
CC(C)(C)c1ccn2->[Ir+]34(<-n5cc(C(F)(F)F)ccc5-c5c(F)cc(F)cc53)(<-n3cc(C(F)(F)F)ccc3-c3c(F)cc(F)cc34)<-n3ccc(C(C)(C)C)cc3-c2c1.CC(C)(C)c1ccnc(-c2cc(C(C)(C)C)ccn2)c1.COCCOC.Cl[Ni]Cl.F[P-](F)(F)(F)(F)F.O=C([O-])[O-].[Cs+]
CC(C)(C)c1ccn2->[Ir+]34(<-n5cc(C(F)(F)F)ccc5-c5c(F)cc(F)cc53)(<-n3cc(C(F)(F)F)ccc3-c3c(F)cc(F)cc34)<-n3ccc(C(C)(C)C)cc3-c2c1.CC(

In [12]:
df.head()

,CATALYST,REACTANT,REAGENT,SOLVENT,INTERNAL_STANDARD,NoData,PRODUCT,YIELD,TEMP
0,CC(C)(C)c1ccn2->[Ir+]34(<-n5cc(C(F)(F)F)ccc5-c...,CC(=O)c1ccc(Br)cc1.O=C(O)C1CCCN1C(=O)OCc1ccccc1,CC(C)(C)c1ccn2->[Ir+]34(<-n5cc(C(F)(F)F)ccc5-c...,CN(C)C=O,,,CC(=O)c1ccc(C2CCCN2C(=O)OCc2ccccc2)cc1,0.93,23.0
1,CC(C)(C)c1ccn2->[Ir+]34(<-n5cc(C(F)(F)F)ccc5-c...,CC(=O)c1ccc(Br)cc1.CC(C)(C)OC(=O)N1CCCCC1C(=O)O,CC(C)(C)c1ccn2->[Ir+]34(<-n5cc(C(F)(F)F)ccc5-c...,CN(C)C=O,,,CC(=O)c1ccc(C2CCCN2C(=O)OCc2ccccc2)cc1,0.82,23.0
2,CC(C)(C)c1ccn2->[Ir+]34(<-n5cc(C(F)(F)F)ccc5-c...,CC(=O)c1ccc(Br)cc1.CC(C)(C)OC(=O)N1CCOCC1C(=O)O,CC(C)(C)c1ccn2->[Ir+]34(<-n5cc(C(F)(F)F)ccc5-c...,CN(C)C=O,,,CC(=O)c1ccc(C2CCCN2C(=O)OCc2ccccc2)cc1,0.61,23.0
3,CC(C)(C)c1ccn2->[Ir+]34(<-n5cc(C(F)(F)F)ccc5-c...,CC(=O)c1ccc(Br)cc1.CC(C)C(NC(=O)OC(C)(C)C)C(=O)O,CC(C)(C)c1ccn2->[Ir+]34(<-n5cc(C(F)(F)F)ccc5-c...,CN(C)C=O,,,CC(=O)c1ccc(C2CCCN2C(=O)OCc2ccccc2)cc1,0.72,23.0
4,CC(C)(C)c1ccn2->[Ir+]34(<-n5cc(C(F)(F)F)ccc5-c...,CC(=O)c1ccc(Br)cc1.CC(C)(C)OC(=O)NC(Cc1cn(C(=O...,CC(C)(C)c1ccn2->[Ir+]34(<-n5cc(C(F)(F)F)ccc5-c...,CN(C)C=O,,,CC(=O)c1ccc(C2CCCN2C(=O)OCc2ccccc2)cc1,0.83,23.0


In [ ]:
df['input'] = 'REACTANT:' + df['REACTANT'] + 'PRODUCT:' + df['PRODUCT'] + 'REAGENT:' + df['REAGENT']
df = df[['input', 'YIELD']].drop_duplicates().reset_index(drop=True)

